In [202]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval

In [203]:
def fixArrays(x):
    start = x.find("[") + len("[")
    end = x.find("]")
    substring = x[start:end].replace("'", "")
    return substring.split(",")

def entitiesMatch(x, y):
    count = 0
    for item1 in x:
        for item2 in y:
            if(item1.strip() == item2.strip()):
                count += 1
                break
    return count

def duplicateElimination(l):
    res = []
    [res.append(x) for x in l if x not in res]
    return res

In [205]:
data = pd.read_csv("datasets/questions_answers_combined.csv", lineterminator='\n', encoding="ISO-8859-1")
data['title_entities'] = data.apply(lambda x: literal_eval(x['title_entities']), axis = 1)
data['question_body_entities'] = data.apply(lambda x: literal_eval(x['question_body_entities']), axis = 1)
data['answer_body_entities'] = data.apply(lambda x: literal_eval(x['answer_body_entities']), axis = 1)
data.head()

,Unnamed: 0,question_user_id,accepted_answer_id,answer_count,question_score,creation_date,question_id,link,title,question_body,...,question_body_entities,answer_id,is_accepted,answer_body,answer_user_id,reputation,answer_score,answer_body_doc,answer_body_token,answer_body_entities
0,0,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,"[PHP, one, A Simple Example: Certain pages on,...",22594,False,<p>While the question touches on a couple of v...,2494.0,17497.0,5,While the question touches on a couple of very...,"['While', 'the', 'question', 'touches', 'on', ...","[User, second, Konrad, OOP, PHP]"
1,1,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,"[PHP, one, A Simple Example: Certain pages on,...",22587,True,<p>These are not really opposite choices. You ...,341.0,17787.0,13,These are not really opposite choices. You wil...,"['These', 'are', 'not', 'really', 'opposite', ...","[PHP3, eCommerce PHP]"
2,2,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,"[PHP, one, A Simple Example: Certain pages on,...",22578,False,<p>Whether you do it in classes or in a more p...,1344.0,21216.0,0,Whether you do it in classes or in a more proc...,"['Whether', 'you', 'do', 'it', 'in', 'classes'...","[three, one, Session, Session, 0, 1, 2, PHPSES..."
3,3,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,"[PHP, one, A Simple Example: Certain pages on,...",22551,False,<p>I've learned never to use <code>include</co...,1968.0,480242.0,1,I've learned never to use include in PHP excep...,"['I', ""'ve"", 'learned', 'never', 'to', 'use', ...","[PHP, one]"
4,4,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,"[PHP, one, A Simple Example: Certain pages on,...",22537,False,<p>Can you be a bit more specific? For the exa...,1830.0,10583.0,0,Can you be a bit more specific? For the exampl...,"['Can', 'you', 'be', 'a', 'bit', 'more', 'spec...","[1, 2, User]"


In [206]:
data['code_snippet_count'] = data.apply(lambda x: x['answer_body'].count('<pre><code>'), axis = 1)
data['link_count'] = data.apply(lambda x: x['answer_body'].count('<a href='), axis = 1)

data.head()

,Unnamed: 0,question_user_id,accepted_answer_id,answer_count,question_score,creation_date,question_id,link,title,question_body,...,is_accepted,answer_body,answer_user_id,reputation,answer_score,answer_body_doc,answer_body_token,answer_body_entities,code_snippet_count,link_count
0,0,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,False,<p>While the question touches on a couple of v...,2494.0,17497.0,5,While the question touches on a couple of very...,"['While', 'the', 'question', 'touches', 'on', ...","[User, second, Konrad, OOP, PHP]",0,0
1,1,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,True,<p>These are not really opposite choices. You ...,341.0,17787.0,13,These are not really opposite choices. You wil...,"['These', 'are', 'not', 'really', 'opposite', ...","[PHP3, eCommerce PHP]",0,1
2,2,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,False,<p>Whether you do it in classes or in a more p...,1344.0,21216.0,0,Whether you do it in classes or in a more proc...,"['Whether', 'you', 'do', 'it', 'in', 'classes'...","[three, one, Session, Session, 0, 1, 2, PHPSES...",1,0
3,3,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,False,<p>I've learned never to use <code>include</co...,1968.0,480242.0,1,I've learned never to use include in PHP excep...,"['I', ""'ve"", 'learned', 'never', 'to', 'use', ...","[PHP, one]",0,0
4,4,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,False,<p>Can you be a bit more specific? For the exa...,1830.0,10583.0,0,Can you be a bit more specific? For the exampl...,"['Can', 'you', 'be', 'a', 'bit', 'more', 'spec...","[1, 2, User]",0,0


In [210]:
data['question_entities'] = data.apply(lambda x: x['title_entities'] + x['question_body_entities'], axis = 1)
data['question_entities'] = data.apply(lambda x: duplicateElimination(x['question_entities']), axis = 1)
data['answer_body_entities'] = data.apply(lambda x: duplicateElimination(x['answer_body_entities']), axis = 1)

data['entities_matches'] = data.apply(lambda x: entitiesMatch(x['question_entities'], x['answer_body_entities']), axis = 1)
data.head()

,Unnamed: 0,question_user_id,accepted_answer_id,answer_count,question_score,creation_date,question_id,link,title,question_body,...,answer_user_id,reputation,answer_score,answer_body_doc,answer_body_token,answer_body_entities,code_snippet_count,link_count,question_entities,entities_matches
0,0,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,2494.0,17497.0,5,While the question touches on a couple of very...,"['While', 'the', 'question', 'touches', 'on', ...","[User, second, Konrad, OOP, PHP]",0,0,"[PHP, one, A Simple Example: Certain pages on,...",1
1,1,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,341.0,17787.0,13,These are not really opposite choices. You wil...,"['These', 'are', 'not', 'really', 'opposite', ...","[PHP3, eCommerce PHP]",0,1,"[PHP, one, A Simple Example: Certain pages on,...",0
2,2,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,1344.0,21216.0,0,Whether you do it in classes or in a more proc...,"['Whether', 'you', 'do', 'it', 'in', 'classes'...","[three, one, Session, 0, 1, 2, PHPSESSID, MVC]",1,0,"[PHP, one, A Simple Example: Certain pages on,...",1
3,3,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,1968.0,480242.0,1,I've learned never to use include in PHP excep...,"['I', ""'ve"", 'learned', 'never', 'to', 'use', ...","[PHP, one]",0,0,"[PHP, one, A Simple Example: Certain pages on,...",2
4,4,2490.0,22587,6,11,2008-08-22,22528,https://stackoverflow.com/questions/22528/php-...,PHP includes vs OOP,<p>I would like to have a reference for the pr...,...,1830.0,10583.0,0,Can you be a bit more specific? For the exampl...,"['Can', 'you', 'be', 'a', 'bit', 'more', 'spec...","[1, 2, User]",0,0,"[PHP, one, A Simple Example: Certain pages on,...",0


In [211]:
data.to_csv('datasets/code_link_features.csv')